In [ ]:
import torch
from torch.nn import functional as F
import torch.nn as nn

In [ ]:
torch.manual_seed(1337)
B,T,C = 4,8,2 
x = torch.randn(B,T,C)

x.shape

In [ ]:
x

In [ ]:
xbow = torch.zeros((B,T,C)) # bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b,:t +1 ] # (t,c)
        xbow [b,t] = torch.mean(xprev,0)

In [ ]:
xbow

In [ ]:
wei = torch.tril((torch.ones(T,T)))
wei = wei / wei.sum(1, keepdim = True) # This produces an average
xbow2 = wei @ x #(T,T) @ (B,T,C) -> toch will identify this dim difference and make it to (B,T,C) @ (B,T,C)----->(B,T,C)
torch.allclose(xbow,xbow2)

In [ ]:
xbow[0], xbow2[0] # Just to observe that they are identical
# Sow e can see that we can get the same value with this trick, and its more computationally efficient that looping

In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
b = torch.randint(0,10,(3,2)).float()

c = a @ b
print("a=")
print(a)
print("b=")
print(b)
print("c=")
print(c)# c will contain the average of the b tensor(row wise, of the above rows). Through this trill trick

In [ ]:
# Third way of building xbow
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros(T,T)
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

In [ ]:
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros(T,T)
wei = wei.masked_fill(tril == 0, float("-inf"))
wei

In [ ]:
wei = F.softmax(wei, dim=1)
wei

In [ ]:
#Self attention
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch time channels
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias=False) # What do I have/know
query = nn.Linear(C, head_size, bias = False) #What am I looking for
value = nn.Linear(C, head_size, bias=False)
k = key(x) # B,T,16
q = query(x) # B,T,16


wei = q @ k.transpose(-2,-1) # (B,T,16) @ (B,16,T) ---> (B,T, T) => For every row of B we have a square matrix with affinities

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros(T,T)
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)


v = value(x) #acts as the information that is stored if the token was interesting

out = wei @ v

out.shape

In [ ]:
wei[0]